# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Additional Notebook (ungraded): Stock Price prediction using ARIMA

## Learning Objectives

At the end of the experiment, you will be able to :

* Understand and preprocess time series data by applying log transformation, exponential decay, and time shift techniques to achieve stationarity.

* Analyze time series dependencies using Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) to determine optimal ARIMA model parameters.

* Build and train an ARIMA model to forecast the demand(passenger traffic) in Airplanes by selecting appropriate values for AR (p), I (d), and MA (q) based on statistical analysis. The data is classified in date/time and the passengers travelling per month

* Evaluate and visualize predictions by applying inverse transformations to recover original scale data and comparing actual vs. forecasted values on a single overlapping plot.


## Information

## Dataset

The Air Passengers dataset is a well-known time series dataset that records the number of passengers traveling by air over time. It consists of monthly observations from January 1949 to December 1960, covering a total period of 12 years (144 months).

**Attributes Information:**

1. Month – Represents the time index in the format YYYY-MM (e.g., 1949-01).
2. Passengers – Represents the total number of passengers (in thousands) who traveled in that particular month.

**Usage in Forecasting:**

- This dataset is used to analyze historical trends in air travel and build a predictive model to forecast future passenger numbers.
- It exhibits characteristics such as seasonality (yearly patterns) and trend (gradual increase over time), making it ideal for time series forecasting using ARIMA.
- Preprocessing steps like log transformation and differencing are applied to make the data stationary before fitting the ARIMA model.

## Problem Statement

Implement ARIMA model to forecast the passenger traffic in Airplanes by using the time series Air Passengers dataset

**contents**
* Import Libraries
* Read Data
* Data Transformation to achieve Stationarity
* Log Scale Transformation
* Exponential Decay Transformation
* Time Shift Transformation
* Plotting ACF & PACF
* Building Models
* Prediction & Reverse transformations

In [ ]:
!pip -qq install pmdarima

In [ ]:
# @title Download the Dataset
! wget https://cdn.exec.talentsprint.com/static/cds/content/Air_Passengers.csv

print("The datset was downloaded")

### Import required Packages

In [ ]:
from datetime import datetime
import numpy as np             #for numerical computations like log,exp,sqrt etc
import pandas as pd           #for reading & storing data, pre-processing
import matplotlib.pylab as plt #for visualization
#for making sure matplotlib plots are generated in Jupyter notebook itself
%matplotlib inline
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima.model import ARIMA
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6

### Load the data and analyze

In [ ]:
df = pd.read_csv('/content/Air_Passengers.csv')
df.head()

In [ ]:
dataset = df
#Parse strings to datetime type
dataset['Month'] = pd.to_datetime(dataset['Month'],infer_datetime_format=True) #convert from string to datetime
indexedDataset = dataset.set_index(['Month'])
indexedDataset.head(5)

In [ ]:
## plot graph
plt.xlabel('Date')
plt.ylabel('Number of air passengers')
plt.plot(indexedDataset)

### Computing Rolling Mean and Standard Deviation for Stationarity Analysis

In [ ]:
#Determine rolling statistics
rolmean = indexedDataset.rolling(window=12).mean() #window size 12 denotes 12 months, giving rolling mean at yearly level
rolstd = indexedDataset.rolling(window=12).std()
print(rolmean,rolstd)

In [ ]:
#Plot rolling statistics
orig = plt.plot(indexedDataset, color='blue', label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

### Augmented Dickey–Fuller test

In [ ]:
#Perform Augmented Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
dftest = adfuller(indexedDataset['#Passengers'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value

print(dfoutput)

From the above result,
1. Test Statistic (1.221233): This value is greater than all the critical values (-3.48, -2.88, -2.57), meaning the null hypothesis (H₀: the series is non-stationary) cannot be rejected.
2. p-value (0.996129): A high p-value (typically > 0.05) suggests that the time series is not stationary, meaning it still exhibits trends or seasonality.
3. Lags Used (14): The model used 14 lag values to estimate stationarity.
4. Number of Observations Used (124): The test was performed on 124 data points after accounting for the lag values.

Since the **p-value is very high (0.996129)**, and the test statistic is above all critical values, the series is still **non-stationary** and requires further transformations (like differencing) to achieve stationarity before applying ARIMA.

### Log Scale Transformation for Trend Estimation

In [ ]:
#Estimating trend
indexedDataset_logScale = np.log(indexedDataset)
plt.plot(indexedDataset_logScale)

### Moving Average Transformation for Stationarity

In [ ]:
#The below transformation is required to make series stationary
movingAverage = indexedDataset_logScale.rolling(window=12).mean()
movingSTD = indexedDataset_logScale.rolling(window=12).std()
plt.plot(indexedDataset_logScale)
plt.plot(movingAverage, color='red')

### Subtracting Moving Average to Detrend the Time Series

In [ ]:
datasetLogScaleMinusMovingAverage = indexedDataset_logScale - movingAverage
datasetLogScaleMinusMovingAverage.head(12)

#Remove NAN values
datasetLogScaleMinusMovingAverage.dropna(inplace=True)
datasetLogScaleMinusMovingAverage.head(10)

### Testing Stationarity using Rolling Statistics and Dickey-Fuller Test

In [ ]:
def test_stationarity(timeseries):

    #Determine rolling statistics
    movingAverage = timeseries.rolling(window=12).mean()
    movingSTD = timeseries.rolling(window=12).std()

    #Plot rolling statistics
    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(movingAverage, color='red', label='Rolling Mean')
    std = plt.plot(movingSTD, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

    #Perform Dickey–Fuller test:
    print('Results of Dickey Fuller Test:')
    dftest = adfuller(timeseries['#Passengers'], autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

test_stationarity(datasetLogScaleMinusMovingAverage)

The above result shows
- Test Statistic: The test statistic is -3.158932. This value is compared to the critical values at different significance levels (1%, 5%, 10%).
- p-value: The p-value is 0.022488, which is less than the previous p-value (0.996129) and less than the common significance level of 0.05. This means we reject the null hypothesis (which suggests the series has a unit root or is non-stationary) and conclude that the series is stationary.

If this result is compared to the previous result where the test statistic was higher (1.221233) and the p-value was 0.996129 which is greater than 0.05, that would indicate non-stationarity.

In the current result, since the test statistic is more negative and the p-value is below 0.05, it indicates that the dataset after subtracting the moving average is stationary.

Hence, the transformation (log scaling and subtracting the moving average) has successfully made the dataset stationary.

**Exponential Weighted Moving Average (EWMA) transformation**

The following code cell applies an Exponential Weighted Moving Average (EWMA) transformation to the time series data to smooth out fluctuations and highlight trends.

The ewm() function is used with a half-life of 12, meaning past observations lose influence at an exponential rate, with each point retaining half of its weight after 12 periods (months).

Unlike a simple moving average, EWMA assigns greater weight to more recent observations, making it more responsive to recent changes in the dataset.
- The min_periods=0 ensures that all available data points are considered without introducing NaN values.
- The adjust=True parameter maintains the default behavior where the EWMA is normalized based on the weights.
- Finally, the original log-transformed dataset is plotted in blue, while the smoothed exponential decay trend is plotted in red to visualize the difference.

In [ ]:
exponentialDecayWeightedAverage = indexedDataset_logScale.ewm(halflife=12, min_periods=0, adjust=True).mean()
plt.plot(indexedDataset_logScale)
plt.plot(exponentialDecayWeightedAverage, color='red')

The following code cell removes the trend from the log-transformed time series by subtracting the Exponential Weighted Moving Average (EWMA) from the original log-scaled data, resulting in a detrended series **(datasetLogScaleMinusExponentialMovingAverage)**.

In [ ]:
datasetLogScaleMinusExponentialMovingAverage = indexedDataset_logScale - exponentialDecayWeightedAverage
test_stationarity(datasetLogScaleMinusExponentialMovingAverage)

From the above result, the following are observed.
- **Test Statistic: -3.541470**, which is more negative than the previous result (-3.158932). So, it suggests stronger evidence for stationarity after subtracting the exponential moving average.
- **p-value: 0.006984**, which is even smaller than the previous p-value of 0.022488. This confirms stronger evidence to reject the null hypothesis and suggests the series is stationary.

The following code cell implements the operation **datasetLogDiffShifting = indexedDataset_logScale - indexedDataset_logScale.shift()**

This is computing the first difference of the log-transformed dataset.

- shift(): This function shifts the values of the time series by one period. Essentially, each value in the series is replaced with the previous value from the series (i.e., it shifts the data by 1).
- Subtraction: By subtracting the shifted series from the original series, the difference between consecutive log-transformed values is calculated. This is done to remove trends and make the series stationary.

In [ ]:
datasetLogDiffShifting = indexedDataset_logScale - indexedDataset_logScale.shift()
plt.plot(datasetLogDiffShifting)

In [ ]:
datasetLogDiffShifting.dropna(inplace=True)
test_stationarity(datasetLogDiffShifting)

### Time Series Decomposition of Log-Scaled Dataset: Trend, Seasonality, and Residuals

In [ ]:
decomposition = seasonal_decompose(indexedDataset_logScale)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(indexedDataset_logScale, label='Original')
plt.legend(loc='best')

plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')

plt.subplot(411)
plt.plot(seasonal, label='Seasonality')
plt.legend(loc='best')

plt.subplot(411)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')

plt.tight_layout()

### ACF and PACF Plots for Differenced Log-Scaled Dataset

In [ ]:
#ACF & PACF plots

lag_acf = acf(datasetLogDiffShifting, nlags=20)
lag_pacf = pacf(datasetLogDiffShifting, nlags=20, method='ols')

plot_acf(lag_acf)
plot_pacf(lag_pacf)

From the above plots, we can get the following.
- Moving Average (q value) = 2 (from acf graph)
- Auto-regressive (p value) = 1 (from pacf graph)
- Differentiation (d value) = 2 (from the window, period value)

We have now removed the ets (error, trend, seasonality)

In [ ]:
# Differencing to make it stationary
df_log_diff = indexedDataset_logScale.diff().dropna()

### ARIMA Model Implementation:
**AR Model Fitting and Visualization on Differenced Log-Scaled Dataset**

In [ ]:
#AR Model
#making order=(2,1,0) gives RSS=1.9972
model = ARIMA(df_log_diff, order=(2,1,0))
results_AR = model.fit()
plt.plot(datasetLogDiffShifting)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues - datasetLogDiffShifting['#Passengers'])**2))
print('Plotting AR model')

In the following code,

The order of the ARIMA model is changed from (2,1,0) to (2,1,2).

**Order (2,1,0):** The previous model has 2 lags for the AR component, 1 differencing step for stationarity, and no MA component (0).

**Order (2,1,2):** The following model still has 2 lags for the AR component and 1 differencing step, but now it includes a Moving Average component with 2 lags (2).

**Why to change the order?**

- Inclusion of the MA component: By adding the MA component (order=2), we are accounting for the residual error (or noise) in the time series, which could help capture more intricate patterns not captured by the AR component alone.
- Model performance: The change in the order helps to improve the model's fit and reduce residuals (RSS).
- By adding the MA component, the model might better account for autocorrelations in the residuals, improving accuracy.

In [ ]:
# AR+I+MA = ARIMA model
model = ARIMA(df_log_diff, order=(2,1,2))
results_ARIMA = model.fit()
plt.plot(datasetLogDiffShifting)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_ARIMA.fittedvalues - datasetLogDiffShifting['#Passengers'])**2))
print('Plotting ARIMA model')

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print(predictions_ARIMA_diff.head())

In [ ]:
#Convert to cumulative sum
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum)

In [ ]:
predictions_ARIMA_log = pd.Series(indexedDataset_logScale['#Passengers'].iloc[0], index=indexedDataset_logScale.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
# Inverse of log is exp.
predictions_ARIMA = np.exp(predictions_ARIMA_log)
print(len(predictions_ARIMA))
plt.plot(indexedDataset)
#plt.plot(indexedDataset_logScale)
plt.plot(predictions_ARIMA)
#plt.plot(predictions_ARIMA_log)

In [ ]:
indexedDataset_logScale

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Forecast the next 120 steps
forecast_steps = 120
forecast_results_log = results_ARIMA.forecast(steps=forecast_steps)

# Convert forecast results back from log scale
forecast_results = np.exp(forecast_results_log)
print(forecast_results)

# Calculate the confidence intervals in log scale
forecast_conf_int_log = results_ARIMA.get_forecast(steps=forecast_steps).conf_int()

# Convert confidence intervals back from log scale
forecast_conf_int = np.exp(forecast_conf_int_log)
print(forecast_conf_int)
# Get confidence intervals and convert from log scale
#forecast_conf_int = np.exp(forecast_results_log.conf_int())

# Ensure forecast starts smoothly from last 'predictions_ARIMA' value
last_date = predictions_ARIMA.index[-1]  # Last available date
forecast_index = pd.date_range(start=last_date, periods=forecast_steps+1, freq='MS')[1:]  # Generate future dates


### Compute Trend Factor for Scaling Forecast

In [ ]:
# **Compute Trend Factor for Scaling Forecast**
recent_trend_factor = predictions_ARIMA.iloc[-12:].pct_change().mean()  # Average monthly growth rate
trend_multiplier = (1 + recent_trend_factor) ** np.arange(1, forecast_steps + 1)  # Apply growth over steps

# Scale the forecasted values using the computed trend multiplier
forecast_results = forecast_results * trend_multiplier * (predictions_ARIMA.iloc[-1] / forecast_results.iloc[0])

# Extract confidence interval bounds
lower_bound = forecast_conf_int.iloc[:, 0] * trend_multiplier * (predictions_ARIMA.iloc[-1] / forecast_conf_int.iloc[:, 0].iloc[0])
upper_bound = forecast_conf_int.iloc[:, 1] * trend_multiplier * (predictions_ARIMA.iloc[-1] / forecast_conf_int.iloc[:, 1].iloc[0])

### Plot ARIMA Forecast for next 120 steps with Confidence Interval

In [ ]:
# **Plot the data**
plt.figure(figsize=(12, 6))

# Original data (blue)
plt.plot(indexedDataset, label='Original Data', color='blue')

# Fitted predictions (orange)
plt.plot(predictions_ARIMA, color='orange', label='Fitted Curve')

# Forecasted values (red)
plt.plot(forecast_index, forecast_results, color='red', linestyle='dashed', label='Forecast (Next 120 Steps)')

# Confidence intervals (shaded region)
plt.fill_between(forecast_index, lower_bound, upper_bound, color='gray', alpha=0.7, label='Confidence Interval')

# Labels and legend
plt.title('ARIMA Forecast with Confidence Interval')
plt.legend()
plt.show()